## Part 1: Script for Component Creation and Pipeline Design

### Setting up the Environment
- Install necessary libraries including `datasets` for data handling, `torch` and `torchdata` for neural network operations, and `transformers`, `evaluate`, `rouge_score`, `loralib`, and `peft` for model training, evaluation, and enhancement.

In [2]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

    
# Installing the Reinforcement Learning library directly from github.
# %pip install git+https://github.com/lvwerra/trl.git@25fa1bd  

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.9 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Note: you may need to restart the kernel to use updated packages.


### Azure ML Workspace Configuration
- Authenticate and initialize Azure Machine Learning client with either `DefaultAzureCredential` or `InteractiveBrowserCredential` for secure access to Azure services.


In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
    
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


### Directory Setup for Scripts
- Create directories for organizing script files and output data, promoting effective project management and data handling.

In [2]:
import os

# create a folder for the script files
script_folder = '../src'
output_folder = '../output'
os.makedirs(script_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)
print(script_folder, 'folder created')

../src folder created


### Sentiment Prediction Pipeline Component Script
- Develop a Python script to predict sentiment from textual dialogue using a pre-trained language model. This script covers model and tokenizer initialization, prompt construction, sentiment prediction, and output handling.


In [16]:
%%writefile $script_folder/sentiment_prediction.py
import mlflow
import argparse
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def predict_sentiment(dialogue, output_path, model_name='google/flan-t5-base'):
    # Initialize tokeniazer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Constructing a 5-shot prompt with examples
    start_prompt = '''Provide Sentiment for the following comment/conversation (possible sentiments: Positive, Negative, Neutral):

    Comment: "I love sunny days, they make me feel so happy!"
    Sentiment: Positive

    Comment: "This is the worst experience of my life, I'm so disappointed."
    Sentiment: Negative

    Comment: "I'm not sure how I feel about this new policy. It might be good or bad."
    Sentiment: Neutral

    Comment: "The service at this restaurant was fantastic, best dinner ever!"
    Sentiment: Positive

    Comment: "I waited for an hour and my order was still wrong."
    Sentiment: Negative

    Comment: '''
    
    end_prompt = '\nSentiment: '
    
    # Construct the full prompt with the user-provided dialogue
    prompt = start_prompt + '"' + dialogue + '"' + end_prompt 

    # Tokenize input dialogue
    inputs = tokenizer(prompt, return_tensors='pt')

    # Generate prediction
    output = model.generate(inputs['input_ids'], max_new_tokens=50)
    
    # Decode and print the prediction
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Write the predicted sentiment to the specified output file
    with open(output_path, 'w') as f:
        f.write('Text: ' + dialogue + '\nPredicted Sentiment: ' + decoded_output + '\n')

def main():
    # enable autologging
    mlflow.autolog()
    
    parser = argparse.ArgumentParser(description="Predict sentiment from input dialogue")
    parser.add_argument("--dialogue", type=str, required=True, help="Input dialogue for sentiment prediction")
    parser.add_argument("--output", type=str, required=True, help="Output file path for sentiment prediction")
    
    args = parser.parse_args()

    # Predict sentiment and write to output
    predict_sentiment(args.dialogue, args.output)

if __name__ == "__main__":
    main()


Overwriting ../src/sentiment_prediction.py


### Testing the Sentiment Prediction Script
- Test the functionality of the sentiment prediction script with a sample dialogue, ensuring accurate sentiment analysis and output generation.


In [39]:
#Test script
!python $script_folder/sentiment_prediction.py --dialogue "I love this book!" --output $output_folder"/output.txt"


2024/03/04 10:30:43 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of transformers. If you encounter errors during autologging, try upgrading / downgrading transformers to a supported version, or try upgrading MLflow.
2024-03-04 10:30:43.872746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 10:30:46.159017: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 10:30:46.811375: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loade

## Part 2: Prepare Resource Scripts: Environment, Compute, and Component Configuration

**Environment Configuration File**  
- Defines necessary dependencies for the sentiment analysis model, ensuring consistent environments for training and inference.

In [ ]:
%%writefile ../src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
  - defaults
dependencies:
  - pip
  - pip:
    - torch==1.13.1
    - rouge_score==0.1.2
  - python=3.10.11
  - datasets==2.17.0
  - torchdata==0.5.1
  - transformers==4.27.2
  - evaluate==0.4.0
  - loralib==0.1.1
  - peft==0.3.0
  - mlflow

**Azure ML Environment Setup**  
- Utilizes the Azure ML SDK to create an environment with specified conda and Docker image settings, facilitating model operations.

In [ ]:
# create new environment using base Docker image and conda specs.
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/curated/acpt-pytorch-1.13-cuda11.7:latest",
    conda_file="../src/conda-env.yml",
    name="docker-image-llm",
    description="llm from docker",
)
ml_client.environments.create_or_update(env_docker_conda)


**Compute Instance Configuration**  
- Configures a compute instance in Azure ML, allocating necessary computational resources for model tasks.

In [ ]:
from azure.ai.ml.entities import ComputeInstance


ci = ComputeInstance(
    name="compute-instance", 
    size="Standard_E4ds_v4"
)
ml_client.begin_create_or_update(ci).result()



**Sentiment Prediction Component Configuration**  
- Creates a YAML configuration for the sentiment prediction component, detailing its execution within Azure ML pipelines, including inputs, outputs, and runtime environment.

In [22]:
%%writefile ../sentiment_prediction.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: sentiment_prediction_merged
display_name: Sentiment Prediction with Integrated Tokenization
version: 3
type: command
inputs:
  dialogue: 
    type: string
outputs:
  sentiment_output:
    type: uri_file
code: ./src
environment: azureml:docker-image-llm@latest
compute: azureml:cpu-cluster
command: >-
  python sentiment_prediction.py 
  --dialogue ${{inputs.dialogue}}
  --output ${{outputs.sentiment_output}}


Overwriting ../sentiment_prediction.yml


## Part 3: Register Component, Configure Pipeline, and Start Command Job

In [ ]:
# from azure.ai.ml import Input
# from azure.ai.ml.constants import AssetTypes
# from azure.ai.ml.dsl import pipeline

# @pipeline()
# def sentiment_prediction(pipeline_job_input):
#     sentiment = predict_sentiment_segment(dialogue=pipeline_job_input)

#     return {
#         "pipeline_job_predict_sentiment_data": sentiment.outputs.sentiment_output,
        
#     }

# # Example usage with a direct string input for the dialogue
# pipeline_job = sentiment_prediction(pipeline_job_input='"Movie is rathet bad!"')
# pipeline_job.settings.default_compute = "compute-instance"

**Component Registration**  
- Utilizes `load_component` to load the sentiment prediction component from YAML and registers it with Azure ML, ensuring the component is versioned and reusable across ML workflows.


In [19]:
from azure.ai.ml import load_component
parent_dir = ""

predict_sentiment_segment = load_component(source=parent_dir + "../sentiment_prediction.yml")

# register component
prep = ml_client.components.create_or_update(predict_sentiment_segment, version='6')

**Pipeline Creation**  
- Defines an ML pipeline using the Azure ML SDK, incorporating the registered sentiment prediction component. This pipeline takes textual input and processes it through the sentiment analysis model, demonstrating how to orchestrate ML tasks.

In [20]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def sentiment_prediction(pipeline_job_input):
    sentiment = predict_sentiment_segment(dialogue=pipeline_job_input)

    return {
        "pipeline_job_predict_sentiment_data": sentiment.outputs.sentiment_output,
        
    }

# Example usage with a direct string input for the dialogue
pipeline_job = sentiment_prediction(pipeline_job_input='"Movie is rathet bad!"')
pipeline_job.settings.default_compute = "compute-instance"

**Pipeline Execution**  
- Submits the configured pipeline as a job to the Azure ML workspace. This step initiates the execution of the sentiment prediction workflow, showcasing how to operationalize ML components.

In [21]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="sentiment_prediction"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
sentiment_prediction,busy_chicken_qw3fty2g06,pipeline,Preparing,Link to Azure Machine Learning studio


**Alternative Pipeline Creation with CLI**  
- Outlines an alternative approach for creating and managing components and pipelines using the Azure CLI. This method provides a script-based option for automating the deployment and execution of ML workflows.

In [ ]:
# #CLI2 version of creating component and pipeline
# !az extension add --name ml -y
# output = %sx az ml component list \
#         --resource-group "cloud-shell-storage-southeastasia" \
#         --workspace-name "oksana_ml"
# print(output)
# !az ml component create --file ../sentiment_prediction.yml
# !az ml job create --file ../pipeline_sentiment_prediction.yml

## Part 4: Implement Online Inference Endpoint and Deploy

### Model Registration with MLflow
- Registers the 'google/flan-t5-base' model for sentiment analysis within MLflow and Azure ML, tailored for future online inference. This process involves saving a pre-trained model and tokenizer, and registering the model under a pyfunc flavour due to specific preprocessing requirements. (testing registration, actual endpoing model to be registered further)

In [25]:
# Script to register the 'google/flan-t5-base' model for sentiment analysis with MLflow and Azure ML for future online inference

# Importing necessary libraries for Azure ML, MLflow, and model loading
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.identity import DefaultAzureCredential
import mlflow
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

def register_model_with_mlflow(model_name='google/flan-t5-base', model_save_path='saved_models/flan_t5_base'):
    """
    Load a pre-trained model and tokenizer from Hugging Face, save them locally,
    and register the model with MLflow for tracking and version control.
    
    Parameters:
    - model_name (str): Identifier for the pre-trained model on Hugging Face.
    - model_save_path (str): Local directory path for saving the model and tokenizer.
    
    Returns:
    - mlflow_model_name (str): The name of the model registered in MLflow, adjusted for compatibility.
    """
    # Adjusting model_name for MLflow compatibility (replacing '/' with '_')
    mlflow_model_name = model_name.replace('/', '_')

    # Loading the tokenizer and model from Hugging Face Transformers
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    
    # Saving the tokenizer and model locally
    tokenizer.save_pretrained(model_save_path)
    model.save_pretrained(model_save_path)
    
    # Logging the model to MLflow with the adjusted name
    mlflow.pytorch.log_model(pytorch_model=model, artifact_path="models", registered_model_name=mlflow_model_name)
    
    return mlflow_model_name

def main():
    # Setting up MLflow: Configuring the tracking URI and experiment name
    mlflow.set_tracking_uri("your_mlflow_tracking_uri")  # Replace with actual MLflow tracking URI
    mlflow.set_experiment("ModelRegistrationExperiment")

    # Starting an MLflow run to log model information
    with mlflow.start_run():
        mlflow_model_name = register_model_with_mlflow()

    # Fetching the latest run from the experiment for artifact URI
    experiment = mlflow.get_experiment_by_name("ModelRegistrationExperiment")
    runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
    latest_run_id = runs.iloc[0]['run_id']  # Assuming the latest run contains the relevant model

    # Extracting the model artifact URI from the latest run information
    run_info = mlflow.get_run(latest_run_id)
    model_artifact_uri = run_info.info.artifact_uri + "/models"


    # Defining Azure ML model metadata for registration
    azure_model = Model(
        name=mlflow_model_name,  # Name of the model in Azure ML registry
        description="Pre-trained sentiment analysis model registered via MLflow.",
        # type="mlflow_model",  # Model type indicating it's tracked by MLflow
        type=AssetTypes.MLFLOW_MODEL,
        path=model_artifact_uri,  # Path to the model artifact for registration
        version="4"  # Model version
    )
    
    # Registering the model in Azure ML
    registered_model = ml_client.models.create_or_update(azure_model)
    print(f"Model registered successfully in Azure ML: {registered_model.name}, Version: {registered_model.version}")

if __name__ == "__main__":
    main()


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'google_flan-t5-base' already exists. Creating a new version of this model...
2024/02/28 08:46:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: google_flan-t5-base, version 12
Created version '12'

Model registered successfully in Azure ML: google_flan-t5-base, Version: 4


### Endpoint Creation
- Defines and creates a managed online endpoint for sentiment analysis, specifying authentication modes and descriptions to set up an accessible and secure endpoint for model inference.

In [3]:
#Define and create an endpoint
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for sentiment extraction",
    auth_mode="key",
)

In [4]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/swagger.json', 'name': 'endpoint-02290657909631', 'description': 'Online endpoint for sentiment extraction', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/resourcegroups/cloud-shell-storage-southeastasia/providers/microsoft.machinelearningservices/workspaces/oksana_ml/onlineendpoints/endpoint-02290657909631', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/providers/Microsoft.MachineLearningServices/locations/australiasoutheast/mfeOperationsStatus/oe:ce49baec-5e26-4e4d-9649-a752220012af:b0f9cdd3-a918-48d8-898d-ffe73bdb7af2?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/sub

Bad pipe message: %s [b'\xe2\xefR\x99Ix\xbe\x92\xde\x1b\xbe\xac}\xa1L\xb9\xe9D \x89t']
Bad pipe message: %s [b"_\xc6\xc9\x00XA\x1b#\x92~\xddqi\xffHUj\x07\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08"]
Bad pipe message: %s [b'\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06']
Bad pipe message: %s [b'', b'\x03\x03']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'', b'\x02

In [3]:
endpoint = ml_client.online_endpoints.get(name = "endpoint-02290657909631")

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/swagger.json', 'name': 'endpoint-02290657909631', 'description': 'Online endpoint for sentiment extraction', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/resourcegroups/cloud-shell-storage-southeastasia/providers/microsoft.machinelearningservices/workspaces/oksana_ml/onlineendpoints/endpoint-02290657909631', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/providers/Microsoft.MachineLearningServices/locations/australiasoutheast/mfeOperationsStatus/oe:ce49baec-5e26-4e4d-9649-a752220012af:b0f9cdd3-a918-48d8-898d-ffe73bdb7af2?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/sub

In [10]:
# #Configure the deployment
# from azure.ai.ml.entities import Model, ManagedOnlineDeployment
# from azure.ai.ml.constants import AssetTypes

# model_name = "google_flan-t5-base"  
# models = ml_client.models.list(name=model_name)
# latest_model = max(models, key=lambda m: m.version)

# blue_deployment = ManagedOnlineDeployment(
#     name="blue",
#     endpoint_name="endpoint-02290657909631",
#     model=latest_model,
#     instance_type="Standard_D2as_v4",
#     instance_count=1,
# )

# #Create deployment
# ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpoint-02290657909631 exists


........................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpoint-02290657909631', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/providers/Microsoft.MachineLearningServices/locations/australiasoutheast/mfeOperationsStatus/od:ce49baec-5e26-4e4d-9649-a752220012af:f4a1c2ad-dc5d-49f6-9516-67c68151b1b4?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/resourceGroups/cloud-shell-storage-southeastasia/providers/Microsoft.MachineLearningServices/workspaces/oksana_ml/onlineEndpoints/endpoint-02290657909631/deployments/blue', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-instance/code/Users/opanasenko2084/Gen.Ai-APP-in-Azure/notebooks', 'creation_context': None, 'serialize': <msr

In [41]:
# # blue deployment takes 100 traffic
# endpoint = ml_client.online_endpoints.get(name = "endpoint-02290657909631")
# endpoint.traffic = {"blue": 100}
# ml_client.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/swagger.json', 'name': 'endpoint-02290657909631', 'description': 'Online endpoint for sentiment extraction', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/resourcegroups/cloud-shell-storage-southeastasia/providers/microsoft.machinelearningservices/workspaces/oksana_ml/onlineendpoints/endpoint-02290657909631', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/providers/Microsoft.MachineLearningServices/locations/australiasoutheast/mfeOperationsStatus/oe:ce49baec-5e26-4e4d-9649-a752220012af:e11d5fac-bda9-48ed-b46e-cd691e507940?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/sub


### Model Preparation for Endpoint
- Develops a custom `SentimentAnalysisModel` under the mlflow.pyfunc flavour to accommodate additional preprocessing like custom prompts and tokenization, specifically designed for online inference scenarios.

In [7]:
%%writefile $script_folder/sentiment_analysis.py
import mlflow.pyfunc
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

class SentimentAnalysisModel(mlflow.pyfunc.PythonModel):
    
    def __init__(self, model_name):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    def predict(self, context, model_input):
        dialogue = model_input.iloc[0]['text']  # Assuming input is a DataFrame with a 'text' column
        prompt = self.construct_prompt(dialogue)
        inputs = self.tokenizer(prompt, return_tensors='pt')
        output = self.model.generate(inputs['input_ids'], max_new_tokens=50)
        decoded_output = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return decoded_output
    
    def construct_prompt(self, dialogue):
        start_prompt = '''Provide Sentiment for the following comment/conversation (possible sentiments: Positive, Negative, Neutral):

        Comment: "I love sunny days, they make me feel so happy!"
        Sentiment: Positive

        Comment: "This is the worst experience of my life, I'm so disappointed."
        Sentiment: Negative

        Comment: "I'm not sure how I feel about this new policy. It might be good or bad."
        Sentiment: Neutral

        Comment: "The service at this restaurant was fantastic, best dinner ever!"
        Sentiment: Positive

        Comment: "I waited for an hour and my order was still wrong."
        Sentiment: Negative

        Comment: '''
        
        end_prompt = '\nSentiment: '
        return start_prompt + '"' + dialogue + '"' + end_prompt


Writing ../src/sentiment_analysis.py


### Environment Minimization
- Streamlines the conda environment to include only essential dependencies, optimizing resource usage and deployment efficiency for the online inference environment.

In [47]:
%%writefile ../src/conda-env-mini.yml
channels:
- conda-forge
dependencies:
- python=3.10.11
- pip<=24.0
- pip:
  - mlflow==2.4
  - cloudpickle==2.2.1
  - torch==1.13.1
  - transformers==4.27.2
name: mlflow-env


Overwriting ../src/conda-env-mini.yml


### MLflow Pyfunc Model Registration
- Logs the `SentimentAnalysisModel` as an MLflow PyFunc model, incorporating example inputs and outputs to infer model signature. This step finalizes the model's preparation for deployment, including registering it with a simplified environment.

In [48]:
import mlflow
from mlflow.models.signature import infer_signature
import pandas as pd


model_name = 'google/flan-t5-base'
sentiment_model = SentimentAnalysisModel(model_name=model_name)

# input and output
example_input = pd.DataFrame({"text": ["This is a great movie!"]})
example_output = pd.DataFrame({"sentiment": ["Positive"]})

# Infer signature using example input and output
signature = infer_signature(example_input, example_output)


# Log the model
with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="sentiment_analysis_model",
        python_model=sentiment_model,
        code_path=["../src/sentiment_analysis.py"],  
        conda_env="../src/conda-env-mini.yml",  
        signature=signature  ,
        registered_model_name = "google_flan-t5-base" #Register model at the same time

    )


Registered model 'google_flan-t5-base' already exists. Creating a new version of this model...
2024/03/04 13:08:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: google_flan-t5-base, version 8
Created version '8' of model 'google_flan-t5-base'.


### Scoring Script Preparation
- Crafts a scoring script (`score.py`) that loads the MLflow model and processes incoming JSON data to perform sentiment analysis, returning structured prediction results. This script acts as the interface between the endpoint and the deployed model.

In [5]:
%%writefile $script_folder/score.py
import json
import pandas as pd
import mlflow.pyfunc

# Path where the MLflow model is saved, adjust as necessary
model_path = "models:/SentimentAnalysisModel/Production"

# Global variable for holding the model
model = None

def init():
    global model
    # Load the MLflow model into the global variable
    model = mlflow.pyfunc.load_model(model_path)

def run(raw_data):
    try:
        # Parse the incoming JSON data
        input_data = json.loads(raw_data)
        
        # Extract the 'text' value to conform with the input signature
        # Creating a DataFrame to match the expected input format for the predict function
        data = pd.DataFrame({"text": [input_data['text']]})
        
        # Use the global model to predict the sentiment
        prediction = model.predict(data)
        
        # Format the output to match the output signature
        # Assuming prediction is returned as a string
        result = {"sentiment": prediction[0]}
        
        # Return the prediction as JSON
        return json.dumps(result)
    except Exception as e:
        error = str(e)
        return json.dumps({"error": error})


Writing ../src/score.py


### Deployment Configuration
- Sets up deployment configurations, including specifying the model, instance type, code configuration, and instance count. This configuration is critical for deploying the model to the online endpoint effectively.

In [9]:
# #Configure the deployment
# from azure.ai.ml.entities import Model, ManagedOnlineDeployment, CodeConfiguration
# from azure.ai.ml.constants import AssetTypes

# model_name = "google_flan-t5-base"  
# models = ml_client.models.list(name=model_name)
# latest_model = max(models, key=lambda m: m.version)


# code_configuration = CodeConfiguration(
#     code="Users/opanasenko2084/Gen.Ai-APP-in-Azure/src",  # Path to the directory containing your 'score.py' and any other necessary files
#     scoring_script="score.py"  # Name of the scoring script
# )

# blue_deployment = ManagedOnlineDeployment(
#     name="blue",  # avoiding green/blue approach and deploying directly to blue production due quota limitation
#     endpoint_name="endpoint-02290657909631",
#     model=latest_model,
#     instance_type="Standard_D2as_v4",
#     instance_count=1,
#     code_configuration=code_configuration  # Add the code configuration to the deployment
# )

In [2]:
#Configure the deployment
from azure.ai.ml.entities import Model, ManagedOnlineDeployment, CodeConfiguration
from azure.ai.ml.constants import AssetTypes

model_name = "google_flan-t5-base"  
models = ml_client.models.list(name=model_name)
latest_model = max(models, key=lambda m: m.version)


code_configuration = CodeConfiguration(
    code="Users/opanasenko2084/Gen.Ai-APP-in-Azure/src",  # Path to the directory containing your 'score.py' and any other necessary files
    scoring_script="score.py"  # Name of the scoring script
)

blue_deployment = ManagedOnlineDeployment(
    name="blue", #avoiding green/blue approach and deploying directly to blue production due quota limitation
    endpoint_name="endpoint-02290657909631",
    model=latest_model,
    instance_type="Standard_D2as_v4",
    instance_count=1,
    code_configuration=code_configuration  # Add the code configuration to the deployment
)



### Deployment and Traffic Management
- Executes the deployment to the managed online endpoint and adjusts traffic allocation to ensure the newly deployed model handles incoming inference requests.

In [ ]:
#Create deployment
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

In [7]:
# blue deployment takes 100 traffic
endpoint_name = "endpoint-02290657909631"
endpoint= ml_client.online_endpoints.get(name=endpoint_name)

endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/swagger.json', 'name': 'endpoint-02290657909631', 'description': 'Online endpoint for sentiment extraction', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/resourcegroups/cloud-shell-storage-southeastasia/providers/microsoft.machinelearningservices/workspaces/oksana_ml/onlineendpoints/endpoint-02290657909631', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/providers/Microsoft.MachineLearningServices/locations/australiasoutheast/mfeOperationsStatus/oe:ce49baec-5e26-4e4d-9649-a752220012af:89a5df1d-4c37-458c-a4f8-6942fcf38a97?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/sub

### Endpoint Testing
- Conducts a test to validate the endpoint's functionality, sending a sample request and verifying the model's ability to return the expected sentiment analysis result.

In [20]:
import json
import tempfile

# Define the endpoint name (change this to your endpoint's name)
endpoint_name = "endpoint-02290657909631"
deployment_name = "blue"
# Construct the scoring URI
scoring_uri = ml_client.online_endpoints.get(name=endpoint_name).scoring_uri


# Define the input data as a Python dictionary
input_data = {
  "input_data": {
    "columns": ["text"],
    "index": [0],
    "data": [
      ["This is the worst customer service experience I've ever had."]
    ]
  },
  "params": {}
}

# Convert the dictionary to a JSON-formatted string
input_json = json.dumps(input_data)

# Create a temporary file to hold the JSON data
with tempfile.NamedTemporaryFile(mode='w', delete=True) as temp_file:
    temp_file.write(input_json)
    temp_file.flush()  # Ensure all data is written to the file

    # Use the temporary file's path to send the scoring request
    response = ml_client.online_endpoints.invoke(
        endpoint_name=endpoint_name,
        deployment_name=deployment_name,
        request_file=temp_file.name  # Use the path to the temporary file
    )

    # The temporary file is automatically deleted here, as we exit the 'with' block

# Print the response from the scoring request
print("Response:", response)


Response: "Negative"


In [ ]:
endpoint.traffic = {"blue": 0}

## Part 6: Batch Version of Endpoint
This section focuses on adapting the sentiment analysis model for batch processing and deploying it as a batch endpoint in Azure ML.

### MLClient Initialization
- Establishes a connection to Azure ML workspace using Azure credentials, preparing for operations like model registration, environment setup, and deployment.

In [5]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
    
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


### Folder Structure Preparation
- Ensures the necessary directory structure is in place for storing scripts and output, facilitating organized code management and execution.

In [4]:
import os

# create a folder for the script files
script_folder = '../src'
output_folder = '../output'
os.makedirs(script_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)
print(script_folder, 'folder created')

../src folder created


### Model Adaptation for Batch Scoring
- Adjusts the sentiment analysis model to handle batch input. This version is designed to process multiple inputs in a single batch, returning a DataFrame of sentiment predictions.

In [7]:
%%writefile $script_folder/sentiment_analysis_batches.py
import mlflow.pyfunc
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

class SentimentAnalysisModel(mlflow.pyfunc.PythonModel):
    
    def __init__(self, model_name):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    def predict(self, context, model_input):
        # Ensure model_input is a DataFrame
        if not isinstance(model_input, pd.DataFrame):
            raise ValueError("Model input must be a pandas DataFrame")
        
        results = []
        for index, row in model_input.iterrows():
            dialogue = row['text']
            prompt = self.construct_prompt(dialogue)
            inputs = self.tokenizer(prompt, return_tensors='pt')
            output = self.model.generate(inputs['input_ids'], max_new_tokens=50)
            decoded_output = self.tokenizer.decode(output[0], skip_special_tokens=True)
            results.append(decoded_output)
        
        # Return a DataFrame with the results
        return pd.DataFrame(results, columns=['sentiment'])
    
    def construct_prompt(self, dialogue):
        start_prompt = '''Provide Sentiment for the following comment/conversation (possible sentiments: Positive, Negative, Neutral):

        Comment: "I love sunny days, they make me feel so happy!"
        Sentiment: Positive

        Comment: "This is the worst experience of my life, I'm so disappointed."
        Sentiment: Negative

        Comment: "I'm not sure how I feel about this new policy. It might be good or bad."
        Sentiment: Neutral

        Comment: "The service at this restaurant was fantastic, best dinner ever!"
        Sentiment: Positive

        Comment: "I waited for an hour and my order was still wrong."
        Sentiment: Negative

        Comment: '''
        
        end_prompt = '\nSentiment: '
        return start_prompt + '"' + dialogue + '"' + end_prompt


Writing ../src/sentiment_analysis_batches.py


### Scoring Script for Batch Processing
- Develops a scoring script capable of handling batch input, transforming JSON data into DataFrames for model prediction, and outputting results in a structured format.

In [6]:
%%writefile $script_folder/score_batches.py

import json
import pandas as pd
import mlflow.pyfunc
import os

# Path where the MLflow model is saved
model_path = "models:/SentimentAnalysisModel/Production"

# Global variable for holding the model
model = None

def init():
    global model
    # Load the MLflow model into the global variable
    model = mlflow.pyfunc.load_model(model_path)

def run(mini_batch):
    # mini_batch is a list of file paths
    results = []
    for file_path in mini_batch:
        # Assuming each file contains data in JSON format
        with open(file_path) as f:
            data = json.load(f)
            # Convert data into DataFrame
            df = pd.DataFrame(data["input_data"]["data"], columns=data["input_data"]["columns"])
            # Perform prediction
            prediction = model.predict(df)
            # Assuming prediction is returned as a list of sentiments
            result = {"sentiment": prediction.tolist()}
            results.append(result)
    
    # Save the results to a file
    output_file_path = os.path.join(os.getenv("AZUREML_BI_OUTPUT_PATH"), "results.json")
    with open(output_file_path, "w") as output_file:
        json.dump(results, output_file)
    
    return output_file_path


Writing ../src/score_batches.py


### Batch Endpoint and Deployment
- Configures and deploys a batch endpoint in Azure ML, defining the model, code configuration, compute resources, and batching settings. This setup allows for efficient processing of large volumes of data, making it suitable for scenarios where real-time inference is not required.

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, Model, CodeConfiguration
from azure.identity import DefaultAzureCredential

# Authenticate to Azure
credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

# Define the model
model_name = "google_flan-t5-base"
models = ml_client.models.list(name=model_name)
latest_model = max(models, key=lambda m: m.version)

# Define the code configuration
code_configuration = CodeConfiguration(
    code="./src",  
    scoring_script="score_batches.py"
)

# Create or update the batch endpoint
batch_endpoint = BatchEndpoint(
    name="batch-endpoint",
    description="Batch endpoint for sentiment analysis",
    tags={"model": "SentimentAnalysisModel"}
)
batch_endpoint = ml_client.begin_create_or_update(batch_endpoint)

# Create the batch deployment
batch_deployment = BatchDeployment(
    name="batch-deployment",
    endpoint_name=batch_endpoint.name,
    model=latest_model.id,
    code_configuration=code_configuration,
    compute="compute-cluster",  # AMLCluster - not created due to azure trial limitations
    mini_batch_size="10",  # Adjust based on your needs
    error_threshold=10,
    retry_settings={"max_retries": 3, "timeout": 30}
)

# Deploy
ml_client.begin_create_or_update(batch_deployment)
